<a href="https://colab.research.google.com/github/amejiyo/Web2Sum/blob/main/TFIDF%20%2B%20TextRank/tfidf%2Btextrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inference (*New*)**

In [ ]:
!pip install thai-segmenter spacy_thai

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import html
from IPython.core.display import display, HTML
from collections import OrderedDict

# Prevent special characters like & and < to cause the browser to display something other than what you intended.
def html_escape(text):
    return html.escape(text)

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai

In [3]:
paragraph = '''วันนอร์โวมัดแน่น อนค.ชี้ปล่อยไม่ได้ นำปท.-ผิดทิศทาง ตู่ ครวญโดนด่าเละ ยอมได้หรือ เปรียบเด็ก นร.ตีกัน ผอ.อย่ามาทะเลาะด้วย เชาว์ ตอก วิษณุ ถกงบฯปี 63 จบใน 105 วัน อย่าตีความวิบัติ ตะแบงใช้ ม.143 ไม่ได้ เทพไท ขย่ม รบ.เหลวปราบทุจริต ดัชนีคอร์รัปชันร่วงรูด 5 อันดับใน 3 ปี บี้แก้ รธน.ปราบโกงให้จริง ฝ่ายค้านเลื่อนเคาะ รมต.เป้าเชือดถึงเย็น 29 ม.ค. สุทิน โอ่ตื่นเต้นข้อมูลใหม่ไหลเข้าไม่หยุด ยังไม่ตัดทิ้งชื่อ บิ๊กป้อม ด้าน วันนอร์ ฟุ้ง ขรก.แห่ส่งหลักฐานเด็ดมัดคอ เสรีพิศุทธ์ จองกฐินชำแหละ บิ๊กตู่ นายกฯ คืนถิ่น รร.เตรียมทหารฯ โอดเร่งแก้สารพัดปัญหา แต่เสียงด่ามาเต็ม อ้อนบิ๊กเอกชนร่วมแรงกันมา 5 ปี อย่าเพิ่งทิ้งกันปัญหา ส.ส.พรรครัฐบาลเสียบบัตรเพื่อกดลงคะแนนแทนกัน ส่งผลกระทบทำให้ร่าง พ.ร.บ.งบประมาณรายจ่ายประจำปี 2563 มีปัญหา ขณะที่นายศรีสุวรรณ จรรยา เลขาธิการสมาคมองค์การพิทักษ์รัฐธรรมนูญไทย ยื่นคำร้องต่อ ป.ป.ช.ให้เอาผิด 3 ส.ส.พรรคภูมิใจไทย และ 1 ส.ส.พรรคพลังประชารัฐที่กดบัตรแทนกัน กรณีกระทำผิดอาจเข้าข่ายการขัดกันแห่งผลประโยชน์ ตามรัฐธรรมนูญ 2560 มาตรา 185 และเข้าข่ายทุจริตต่อหน้าที่ตาม พ.ร.บ.ป้องกันและปราบปรามการทุจริต พ.ศ.2561 และเข้าข่ายฝ่าฝืนมาตรฐานทางจริยธรรมอย่างร้ายแรงหรือไม่ศรีสุวรรณ ยื่นสอบ 4 ส.ส.กดบัตรแทนกันเมื่อเวลา 10.30 น. วันที่ 27 ม.ค. ที่สำนักงานคณะกรรมการป้องกันและปราบปรามการทุจริตแห่งชาติ (ป.ป.ช.) นายศรีสุวรรณ จรรยา เลขาธิการสมาคมองค์การพิทักษ์รัฐธรรมนูญไทย ยื่นคำร้องต่อ ป.ป.ช.ให้เอาผิด 3 ส.ส.พรรคภูมิใจไทย ได้แก่ นายฉลอง เทอดวีระวงศ์ ส.ส.พัทลุง นางนาที รัชกิจประการ ส.ส.บัญชีรายชื่อ นายสมบูรณ์ ซารัมย์ ส.ส.บุรีรัมย์ รวมถึง ส.ส.พรรคพลังประชารัฐ 1 คน ได้แก่ น.ส.ภริม พูลเจริญ ส.ส.สมุทรปราการ กรณีกระทำผิดอาจเข้าข่ายการขัดกันแห่งผลประโยชน์ ตามรัฐธรรมนูญ 2560 มาตรา 185 โดยกรณีนายฉลองและนางนาทีนั้นพบว่ามีชื่อเป็นผู้ลงคะแนนเห็นชอบร่าง พ.ร.บ.งบฯปี 63 เมื่อวันที่ 11 ม.ค.63 ทั้งที่เจ้าตัวไม่ได้อยู่ในห้องประชุมในวันดังกล่าว ขณะที่ น.ส.ภริมและนายสมบูรณ์พบพฤติการณ์เสียบบัตรลงคะแนนแทนกัน โดยนำบัตรลงคะแนนของผู้อื่นมากดลงมติให้ตามที่ปรากฏในคลิปของสื่อมวลชน จนนำไปสู่การยื่นให้ศาลรัฐธรรมนูญวินิจฉัยว่าร่าง พ.ร.บ.ดังกล่าวตราขึ้นโดยไม่ถูกต้องตามรัฐธรรมนูญหรือไม่ซัดทุจริตต่อหน้าที่–ผิดจริยธรรมร้ายแรงนายศรีสุวรรณกล่าวว่า อยากให้ ป.ป.ช.ตรวจสอบการกระทำว่าเข้าข่ายทุจริตต่อหน้าที่ตาม พ.ร.บ.ป้องกันและปราบปรามการทุจริต พ.ศ.2561 หรือไม่ รวมถึงเข้าข่ายฝ่าฝืนมาตรฐานทางจริยธรรมอย่างร้ายแรง ต้องถือผลประโยชน์ของประเทศเหนือกว่าประโยชน์ส่วนตน และต้องไม่มีพฤติการณ์ที่รู้เห็นหรือยินยอมให้ผู้อื่นใช้ตำแหน่งหน้าที่แสวงหาประโยชน์โดยมิชอบ หาก ป.ป.ช.วินิจฉัยว่ามีความผิดตามข้อห้ามเหล่านี้ ให้ยื่นฟ้องต่อศาลฎีกาของผู้ดำรงตำแหน่งทางการเมืองพิจารณานำไปสู่การสิ้นสุดลงของตำแหน่ง ส.ส. ตามมาตรา 101 (7) ของรัฐธรรมนูญปี 2560 และอาจมีความผิดตาม พ.ร.บ.ป้องกันและปราบปรามการทุจริต พ.ศ.2561 มาตรา 28 (1) และมาตรา 30 วรรคแรก เพื่อมิให้เป็นเยี่ยงอย่างที่ไม่ชอบด้วยกฎหมายต่อไปอนุทิน โยน ปชป.คุมกันเองมารยาทมีอยู่เมื่อเวลา 10.00 น. ที่ทำเนียบรัฐบาล นายอนุทิน ชาญวีรกูล รองนายกฯและ รมว.สาธารณสุข ในฐานะหัวหน้าพรรคภูมิใจไทย กล่าวกรณีนายนิพิฏฐ์ อินทรสมบัติ รองหัวหน้าพรรคประชาธิปัตย์ เปิดประเด็นกรณี ส.ส.พรรคภูมิใจไทย เสียบบัตรแทนกันในการลงมติร่าง พ.ร.บ.งบประมาณรายจ่ายประจำปี 2563 ได้พูดคุยทำความเข้าใจกับพรรคประชาธิปัตย์เกี่ยวกับเรื่องดังกล่าวหรือยังว่า เป็นเรื่องของพรรคประชาธิปัตย์ เรื่องนี้คงไม่ต้องพูดคุยอะไร คนที่เป็นต้นสังกัดเป็นหัวหน้าต้องไปจัดการเรื่องภายในพรรคของตัวเอง เพราะมารยาทในการอยู่ร่วมมีอยู่แล้ว ทำงานร่วมกันต้องพยายามขัดแย้งให้น้อยที่สุด ทำความเข้าใจให้มากที่สุด ถ้าคนในสังกัดมีกิริยาที่ไม่เหมาะสมไม่ถูกต้อง และทำให้การร่วมมือในการทำงานร่วมกันมีอุปสรรคคนที่เป็นต้นสังกัดก็ต้องไปจัดการ เหมือนที่ตนจัดการกับคนในพรรคของตน ผู้สื่อข่าวถามว่า จากเรื่องดังกล่าวพรรคภูมิใจไทยกับพรรคประชาธิปัตย์ยังสามารถทำงานร่วมกันต่อไปได้หรือไม่ นายอนุทินกล่าวว่า ได้ไม่มีปัญหาเราเอาเรื่องบ้านเมืองเป็นเรื่องสำคัญนิพิฏฐ์ ถามโกง ปชต. ยอมได้หรือนายนิพิฏฐ์ อินทรสมบัติ รองหัวหน้าพรรคประชาธิปัตย์ โพสต์เฟซบุ๊กว่า ถ้าเรายอมให้เกิดการโกงในกระบวนการประชาธิปไตย แล้วครั้งต่อไปล่ะ ครั้งต่อไปอีกล่ะ เราจะเอาไงหรือเราจะบอกว่า ครั้งนี้เรายอมให้โกงได้ เพราะอะไรล่ะ ตรรกะวิบัติหรือเปล่า? นายนิพิฏฐ์ให้สัมภาษณ์เพิ่มเติมว่าที่โพสต์ไปว่าทำไปเพราะถือว่าเตือนสติคนบางคนให้คิดได้หรือคิดให้ตก เพราะยังมีบางคนวิจารณ์ว่าแม้จะผิดกฎหมายแต่ไม่ควรพูดเช่นนั้น จึงใช้สื่อโซเชียลโพสต์ชี้แจงเพื่อให้เห็นถึงตรรกะหรือวิธีคิดว่าคิดแบบนี้ไม่ถูกต้อง ผิดเพี้ยนจากหลักการความถูกต้องและข้อเท็จจริงที่เกิดขึ้น ที่เคยระบุเรียกร้องให้ 2 ส.ส.มารับผิดว่าไม่ได้อยู่ประชุมการพิจารณาร่าง พ.ร.บ.งบฯ ปี 63 มาตราใดบ้าง แล้วยังเงียบอยู่ก็เป็นเรื่องของเขา หากคิดว่าจะระงับความเสียหายที่เกิดขึ้นควรไปให้ข้อเท็จจริงกับศาลรัฐธรรมนูญว่าทั้งสองไม่ได้โหวตในมาตราใด แต่หากจะเงียบเป็นคลื่นกระทบฝั่งก็เป็นเรื่องของเขาที่จะคิดและทำเช่นนั้นปัดขัดแย้ง นร.ตีกัน ผอ.อย่าขยายวงเมื่อถามว่า นายอนุทิน ชาญวีรกูล รมว.สาธารณสุขและหัวหน้าพรรคภูมิใจไทยระบุให้หัวหน้าพรรคประชาธิปัตย์ไปจัดการเรื่องในพรรค เพราะมารยาทการอยู่ร่วมกันต้องขัดแย้งให้น้อยที่สุด นายนิพิฏฐ์กล่าวว่า ตนทำในนามส่วนตัวที่ทราบข้อมูลเรื่องการทำผิดกฎหมายรัฐธรรมนูญ นำเรื่องดังกล่าวมาแถลงต่อสาธารณชน โดยไม่ได้ปรึกษาใครในพรรคประชาธิปัตย์ และการแถลงดังกล่าวถือเป็นการช่วยให้กฎหมายมีความสมบูรณ์เสียด้วยซ้ำ ดังนั้นเรื่องนี้จึงไม่ควรขยายวงกลายเป็นความขัดแย้งในพรรคร่วมรัฐบาล เพราะหากเปรียบไปเป็นเรื่องของบุคคล หรือนักเรียนสองโรงเรียนสองคนที่ทะเลาะกัน ดังนั้น ผอ.ของสองโรงเรียนจึงไม่ควรลงมาทะเลาะกันด้วย เพียงแต่เรื่องนี้เป็นเรื่องส่วนตัวที่เกี่ยวข้องกับกฎหมาย จึงต้องดูถึงสาเหตุของการวิวาทของนักเรียนว่าต้นเหตุคืออะไร ส่วนตนมีความเคารพหัวหน้าพรรคภูมิใจไทยและไม่มีอะไรที่จะตอบโต้ได้แต่ชี้แจงข้อเท็จจริงไปตามเนื้อผ้าเชาว์ ซัดศรีธนญชัยซิกแซก ลอดล่องเมื่อเวลา 15.00 น. นายเชาว์ มีขวด อดีตรองโฆษกพรรคประชาธิปัตย์และทนายอาสาโพสต์เฟซบุ๊กหัวข้อ ชำแหละ ม.143 เมื่อนักกฎหมายกลายเป็นศรีธนญชัย ลอดล่อง ว่า เห็นข่าวนายวิษณุ เครืองาม รองนายกฯให้ความเห็นด้านกฎหมายเกี่ยวกับทางออก หากศาลรัฐธรรมนูญวินิจฉัยให้ร่าง พ.ร.บ.งบประมาณรายจ่ายประจำปี 2563 เป็นโมฆะไว้ว่าในมาตรา 143 ของรัฐธรรมนูญเขียนไว้ชัดว่าหากพิจารณาไม่แล้วเสร็จใน 105 วัน ให้ถือว่าสภาฯเห็นชอบซึ่งกฎหมายงบประมาณฯ ถ้าไม่ล็อกเรื่อง 105 วัน มันมีช่องทางคิดได้เหมือนกันว่าเอากลับไปโหวตใหม่ แต่เมื่อมีกำหนดเวลาเอาไว้ชัด เป็นช่องขอให้ศาลวินิจฉัยว่าจะเอามาใช้ประโยชน์ได้หรือไม่ หากใช้ประโยชน์ได้มันจะกลับไปสู่ร่าง พ.ร.บ.งบฯปี 63 ที่เสนอวาระ 1 ทุกอย่างที่คณะกมธ.วิสามัญพิจารณาร่าง พ.ร.บ.งบฯปี 63 ตัดไปจะกลับไปสู่ร่างแรก เพราะเจตนามาตรานี้ต้องการให้เป็นไปอย่างนั้นอยู่แล้ว แต่ไม่ได้คิดว่าจะเกิดกระบวนการทำผิดหรือคณะ กมธ.ทำล่าช้า ถ้าเป็นไปตามที่นายวิษณุที่สังคมให้สมญานามว่า ศรีธนญชัย พยายามหาช่องให้กลับไปใช้ร่างเดิม ส่งต่อให้สภาทหารเกณฑ์โหวตเห็นชอบได้ภายใน 1 วัน ย่นระยะเวลาให้เร็วขึ้นงบฯจบใน 105 วัน วิบัติตะแบงใช้ ม.143ในฐานะนักกฎหมายคิดว่าช่องนี้ลอดไม่ได้และไม่ควรลอด เพราะข้อเท็จจริงชัดเจนว่ากระบวนการพิจารณาของสภาฯนับตั้งแต่รับร่าง พ.ร.บ.งบฯ 63 มาจนลงมติในวาระสามเป็นไปตามกรอบเวลา 105วันที่รัฐธรรมนูญกำหนด ปัญหาที่เกิดขึ้นเป็นกรณีการลงคะแนนที่ไม่สุจริต มีการกดบัตรแทนกัน จึงไม่มีเหตุผลใดที่จะรับฟังว่าเมื่อลงคะแนนไม่สุจริตเท่ากับสภาฯพิจารณาไม่แล้วเสร็จภายใน 105 วัน อย่างที่นายวิษณุพยายามตะแบง การพยายามใช้มาตรา 143 มาเพิ่มทางรอดให้รัฐบาล เท่ากับกำลังตีความกฎหมายตามอำเภอใจเพื่อประโยชน์ตัวเอง ถือเป็นความวิบัติในการใช้ดุลพินิจทางกฎหมายที่จะทำให้เกิดปัญหาอื่นตามมา อย่าลากศาลรัฐธรรมนูญมาเป็นเกราะกำบังให้รัฐบาลเลย ให้แต่ละฝ่ายทำหน้าที่ถ่วงดุลอำนาจกันและกันตามหลักประชาธิปไตย ให้อำนาจตุลาการเป็นเสาหลักค้ำยันประชาธิปไตยได้อย่างมั่นคงต่อไปก.ม.ไม่ได้มีไว้รับใช้ความอยุติธรรมยิ่งไปกว่านั้นคำร้องวิปรัฐบาลที่ขอให้ศาลรัฐธรรมนูญแนะนำว่าถ้าร่างงบฯปี 63 ตกไปทั้งฉบับหรือบางมาตราต้องดำเนินการอย่างไร ไม่สอดคล้องกับอำนาจหน้าที่ศาลรัฐธรรมนูญ ที่มาตรา 210 (1) ให้ศาลวินิจฉัยความชอบด้วยรัฐธรรมนูญของกฎหมายหรือร่างกฎหมาย ศาลรัฐธรรมนูญไม่ได้เป็นที่ปรึกษากฎหมายของรัฐบาล คำร้องดังกล่าวจึงไม่อยู่ในขอบเขตอำนาจของศาลรัฐธรรมนูญจะวินิจฉัยหรือตอบเรื่องนี้ จึงขอยืมคำพูดนายนิพิฏฐ์ อินทรสมบัติ รองหัวหน้าพรรคประชาธิปัตย์ ที่บอกว่ากฎหมายต้องรับใช้ความยุติธรรม ไม่ใช่นำกฎหมายไปรับใช้ความอยุติธรรม เมื่อไหร่ก็ตามที่เราปล่อยให้มีการบิดเบือนกฎหมาย เพื่อประโยชน์ของตัวเองและพวกพ้อง เมื่อนั้นสังคมจะหาความสงบไม่ได้ เหมือนที่เคยเกิดขึ้นมาแล้วในยุคนายทักษิณเรืองอำนาจ จึงฝากถึงนายวิษณุว่าสมควรทบทวนบทบาทตนเอง สังคมให้สมญานามว่าศรีธนญชัยหนักอยู่แล้ว ถ้าฝืนเดินต่อไปรอดร่องตกบันไดขาหักขึ้นมา ไม่รู้จะตั้งฉายาว่าอย่างไร นายเชาว์กล่าวเทพไท ขย่มรัฐเหลวปราบโกงไทยร่วงนายเทพไท เสนพงศ์ ส.ส.นครศรีธรรมราช พรรคประชาธิปัตย์ กล่าวถึงกรณีองค์กรเพื่อความโปร่งใสนานาชาติ ประกาศดัชนีความโปร่งใสหรือดัชนีคอร์รัปชันล่าสุด ประจำปี 2562 ประเทศไทยร่วงไปอยู่อันดับ 101 จาก 180 ประเทศทั่วโลกว่ารัฐบาลและองค์กรที่เกี่ยวข้องกับการป้องกันปราบปรามการทุจริตต้องตระหนักและเร่งปรับปรุงการทำงานการตรวจสอบและปราบปรามการทุจริตให้รัดกุม รอบคอบ จริงจังมากขึ้น ดัชนีความโปร่งใสของประเทศตกต่ำลงมาต่อเนื่อง ปี 60 อันดับที่ 96 ปี 61 อันดับที่ 99 ปี 62 อันดับที่ 101 ได้ 36 เต็ม 100 คะแนน ถดถอยลงมา 5 อันดับในรอบ 3 ปี แสดงให้เห็นถึงความล้มเหลวเรื่องการป้องกันและปราบปรามการทุจริตของรัฐบาลชุดนี้ นับตั้งแต่ คสช.มาควบคุมอำนาจบริหารประเทศ มีข้ออ้างสาเหตุการทุจริตของนักการเมืองจากการเลือกตั้ง แต่เมื่อ คสช.บริหารประเทศเอง การทุจริตคอร์รัปชันไม่มีแนวโน้มจะลดลง กลับแพร่กระจายไปอย่างกว้างขวาง ไม่มีการตรวจสอบถ่วงดุลใดๆทั้งสิ้นกระทุ้งแก้ รธน.ปราบโกงให้จริงกระทั่ง คสช.ได้แต่งตั้งคณะกรรมาธิการร่างรัฐธรรมนูญ (กรธ.) เพื่อยกร่างรัฐธรรมนูญขึ้นมาใหม่ ประกาศว่าเป็นรัฐธรรมนูญฉบับปราบโกง สร้างความหวังให้ผู้คนในสังคมที่ต้องการเห็นความเปลี่ยนแปลงของบ้านเมือง อยากเห็นประเทศไทยใสสะอาด ปราศจากการทุจริตคอร์รัปชัน แต่ในที่สุดหลังประกาศใช้รัฐธรรมนูญฉบับนี้มาแล้ว 3 ปี ดัชนีความโปร่งใสของประเทศถดถอยลง 3 ปีติดต่อ เป็นบทพิสูจน์เห็นได้ชัดว่ารัฐธรรมนูญฉบับปราบโกงเป็นเพียงแค่วาทกรรมหรือการโฆษณาชวนเชื่อ หวังผลให้ประชาชนคนไทยลงมติรับร่างรัฐธรรมนูญฉบับนี้เท่านั้น สร้างกระแสการปราบโกงกลบเกลื่อน เบี่ยงเบนกระแสการสืบทอดอำนาจ คสช.ที่กำลังถูกต่อต้านอย่างหนัก ถึงเวลาแล้วจำเป็นต้องรื้อโครงสร้างรัฐธรรมนูญเกี่ยวกับกลไกปราบโกงและประเด็นที่ไม่เป็นประชาธิปไตย จำเป็นยิ่งต้องสนับสนุนให้คณะกมธ.วิสามัญศึกษาหลักเกณฑ์และวิธีการแก้ไขรัฐธรรมนูญฉบับนี้ ศึกษาจุดอ่อนของรัฐธรรมนูญ ต้องทบทวนหมวดเกี่ยวกับองค์กรอิสระปราบปรามการทุจริตคอร์รัปชัน และโครงสร้างการตรวจสอบถ่วงดุลขององค์กรอิสระ เพิ่มการตรวจสอบภาคประชาชนให้เข้มแข็งขึ้น เพื่อเป็นเครื่องมือปราบปรามการทุจริต เรียกร้องให้รัฐบาลใช้โอกาสนี้ปรับปรุงกวดขัน ตรวจสอบการทำงานของทุกภาคส่วนเกี่ยวกับการปราบปรามการทุจริตคอร์รัปชัน ทำให้ดัชนีความโปร่งใสของประเทศอยู่ในลำดับดีขึ้น จะได้ไม่อับอายต่อประชาคมโลกอีกต่อไป นายเทพไทกล่าวฝ่ายค้านคุยข้อมูลเชือดไหลเข้าไม่หยุดเมื่อเวลา 10.00 น. ที่พรรคเพื่อไทย นายสุทิน คลังแสง ประธานวิปฝ่ายค้าน กล่าวถึงการยื่นญัตติอภิปรายไม่ไว้วางใจรัฐบาล พล.อ.ประยุทธ์ จันทร์โอชา ว่า วิปฝ่ายค้านพร้อมเต็มที่ เชื่อว่าจะยื่นญัตติในวันที่ 29 ม.ค. รัฐมนตรีที่จะถูกอภิปรายไม่ไว้วางใจในส่วนของพรรคเพื่อไทยเป็นคนกลุ่มเดียวกันที่พรรคร่วมฝ่ายค้านเตรียมยื่นญัตติหรือจะเป็นตัวเลข 5+4 ซึ่ง 4 คนหลังอาจจะเพิ่มหรือลดในภายหลัง จะกลั่นกรองรายละเอียดขั้นสุดท้าย เนื่องจากมีข้อมูลที่จะนำไปอภิปรายใหม่ๆอยู่ มีทั้งปัญหาฝุ่นพีเอ็ม 2.5 มาตรการป้องกันไวรัสโคโรนาสายพันธุ์ใหม่ และเรื่องในสภาฯที่สะท้อนถึงภาวะผู้นำในการแก้ไขปัญหาของประเทศ ส่วนที่โพลบางสำนักต้องการให้ฝ่ายค้านอภิปรายรัฐบาลทั้งคณะ พร้อมรับฟัง แต่มีข้อดีและข้อเสียต่างกัน การอภิปรายนายกฯหมายรวมถึงการอภิปรายคณะรัฐมนตรีทั้งคณะ แล้วการอภิปรายทั้งคณะจะกระทบกับคนที่ทำงานดี ส่วนการอภิปรายรัฐมนตรีเฉพาะตัวบุคคล จะชี้ชัดการทำงานได้มากกว่าว่าใครคือจุดอ่อนที่ต้องสกัด หรือใครคือจุดแข็งที่ต้องรักษาไว้เชื่อกฎหมายงบประมาณเป็นโมฆะนายสุทิน ยังกล่าวถึงคำร้องที่ยื่นต่อศาลรัฐธรรมนูญวินิจฉัยร่าง พ.ร.บ.งบประมาณว่า คำร้องได้ยื่นไปใน 2 ประเด็นคือ ความผิดในตัวบุคคลและสถานะของร่างกฎหมายฉบับนี้ โดยส่วนตัวเชื่อว่าร่างกฎหมายฉบับนี้จะเป็นโมฆะ หากศาลรัฐธรรมนูญชี้ว่าร่างกฎหมายนี้ตกไป ต้องหาทางออกกันต่อไป โดยจะมีการหารือร่วมกันในพรรคฝ่ายค้าน ทั้งนี้สำหรับแนวทางการแก้ปัญหาการเสียบบัตรแทนกันของ ส.ส.นั้นในอนาคต เมื่อห้องประชุมใหญ่แล้วเสร็จ จะมีการระบุที่นั่งของ ส.ส.แต่ละคน จึงทำให้ไม่สามารถเสียบบัตรแทนกันได้ และเชื่อว่าการลงโทษ ส.ส.ในครั้งนี้จะเป็นบทเรียนให้กับ ส.ส.ในสภาฯสมพงษ์ ชี้เลื่อนยื่นญัตติไม่กระทบต่อมาเวลา 11.30 น. ที่พรรคเพื่อไทย นายสมพงษ์ อมรวิวัฒน์ หัวหน้าพรรคเพื่อไทยและผู้นำฝ่ายค้าน นายภูมิธรรม เวชยชัย ที่ปรึกษาผู้นำฝ่ายค้าน น.อ.อนุดิษฐ์ นาครทรรพ เลขาธิการพรรคเพื่อไทย นายสุทิน คลังแสง ประธานวิปฝ่ายค้าน นายธนาธร จึงรุ่งเรืองกิจ หัวหน้าพรรคอนาคตใหม่ นายวันมูหะมัดนอร์ มะทา หัวหน้าพรรคประชาชาติ นพ.เรวัต วิศรุตเวช ส.ส.บัญชีรายชื่อพรรคเสรีรวมไทย นายสงคราม กิจเลิศไพโรจน์ หัวหน้าพรรคเพื่อชาติ นายนิคม บุญวิเศษ หัวหน้าพรรคพลังปวงชนไทย ร่วมประชุมเพื่อกำหนดกรอบประเด็นและรัฐมนตรีที่จะถูกยื่นญัตติอภิปรายไม่ไว้วางใจ โดยนายสมพงษ์ กล่าวในที่ประชุมว่า พรรคร่วมฝ่ายค้านที่จะยื่นญัตติอภิปรายไม่ไว้วางใจรัฐบาล เดิมกำหนดไว้วันที่ 29 ม.ค. แต่ถ้าหากจะมีการเลื่อนออกไปจากเดิม 1-2 วัน ไม่น่ามีปัญหา และคงจะไม่กระทบกับกรอบวันของการอภิปรายไม่ไว้วางใจฉะให้ยับ รบ.ไร้น้ำยาบริหารประเทศนายภูมิธรรมกล่าวว่า การอภิปรายครั้งนี้พรรคร่วมฝ่ายค้านตั้งใจให้ประชาชนเข้ามามีส่วนร่วมตรวจสอบรัฐบาลด้วย ไม่ได้หวังพึ่งมือในสภาฯเพียงอย่างเดียว รัฐบาลแม้จะเป็นเสียงข้างมาก แต่อยู่ในภาวะเสียงปริ่มน้ำ ฝ่ายค้านจะชี้ให้ประชาชนได้ประจักษ์ว่า การบริหารงานที่ผ่านมาของรัฐบาล ก่อให้เกิดประสิทธิภาพหรือไม่ ทั้ง 7 พรรคร่วมฝ่ายค้าน มีความพร้อมมีความเข้าใจร่วมกัน การประชุมครั้งนี้จะได้ร่วมหารือถึงบุคคลที่จะถูกอภิปรายไม่ไว้วางใจ ชื่อที่เสนอมาซ้ำกันหรือไม่ คงจะแลกเปลี่ยนพูดคุยตกลงกัน เพื่อให้การอภิปรายมีประสิทธิภาพยิ่งขึ้นธนาธร ปล่อยไม่ได้นำ ปท.ผิดทิศทางนายธนาธรกล่าวว่า พรรคอนาคตใหม่ยินดีรับกรอบที่พรรคร่วมฝ่ายค้านที่ได้หารือร่วมกันที่จะร่วมอภิปรายถึงความล้มเหลวของรัฐบาลนี้ นำพาประเทศไปผิดทิศผิดทาง เราจะร่วมทำหน้าที่แทน ประชาชน ตรวจสอบการทำงานรัฐบาล กรอบที่ฝ่ายค้านได้คุยกันมีทั้งปัญหาเศรษฐกิจ ปัญหาสังคม ประเด็นที่อาจจะมีการทุจริตคอร์รัปชัน พรรคร่วมฝ่ายค้านจะชี้ให้เห็นถึงความไม่มีประสิทธิภาพ ความล้มเหลวของรัฐบาลนี้ ขณะเดียวกัน ขอให้ประชาชนและฝ่ายรัฐบาลช่วยกันร่วมตรวจสอบสิ่งที่ฝ่ายค้านได้นำเสนอ ถ้าเปิดใจกว้างรับฟังอย่างไม่มีอคติ จะรู้ว่า เราไม่อาจไว้วางใจให้รัฐบาล พล.อ.ประยุทธ์ ได้บริหารประเทศต่อไปวันนอร์ ฟุ้ง ขรก.แห่ส่งหลักฐานเด็ดนายวันมูหะมัดนอร์ มะทา หัวหน้าพรรคประชาชาติ กล่าวว่า พรรคจะร่วมตรวจสอบประเด็นการบริหารงานที่ขาดจริยธรรมในหลายประการ มีทั้งหลักฐานที่จับต้องได้ เพราะเราได้รับข้อมูลจากประชาชน ฝ่ายราชการเข้ามาเยอะมาก ขณะเดียวกันหลังยึดอำนาจเมื่อปี 2557 รัฐบาลระบุจะปฏิรูปประเทศในหลายด้าน แต่กลับไม่ได้มีการปฏิรูปอะไรเลย ถือเป็นความล้มเหลวอย่างสิ้นเชิง เราจะ ชี้ให้เห็นสิ่งที่รัฐบาลพูดอย่างทำอย่าง ขอให้ประชาชนร่วมติดตาม ฝ่ายค้านจะพูดในสิ่งที่เป็นจริง มีประโยชน์ มีข้อเท็จจริงที่จับต้องได้เสรีพิศุทธ์ จองกฐินชำแหละ บิ๊กตู่นพ.เรวัตกล่าวว่า ได้รับแจ้งจาก พล.ต.อ.เสรีพิศุทธ์ เตมียเวส หัวหน้าพรรคเสรีรวมไทย จะอภิปรายนายกฯ นายวิษณุ เครืองาม รองนายกฯ ร.อ.ธรรมนัส พรหมเผ่า รมช.เกษตรฯ โดยเรื่องของ นายกฯคือการถวายสัตย์ฯไม่ครบถ้วน นอกจาก พล.ต.อ.เสรีพิศุทธ์ที่จะอภิปรายแล้ว ยังมีตนที่จะชี้ให้เห็นตั้งแต่การบริหารงานของประยุทธ์ 1 มาถึง การสืบทอดอำนาจจนมีรัฐบาลประยุทธ์ 2 ในการตั้ง ส.ว.แล้วคนเหล่านั้นก็มาโหวตเลือกให้ได้เป็นนายกฯอีกครั้ง ไม่มีประเทศไหนในโลกทำกัน นอกจากนี้ ยังจะพูดถึงการทำงานของรัฐบาลที่เอื้อให้กับบริษัททุนขนาดใหญ่ โดยเจาะลงไปที่โครงการอีอีซี โดย มีการใช้อำนาจตามมาตรา 44เย้ยโซเชียลรุมสวด รบ.ไร้กึ๋นนายอนุสรณ์ เอี่ยมสะอาด โฆษกพรรคเพื่อไทย แถลงว่า ถ้าวิเคราะห์ข่าวของรัฐบาลในช่วงที่ผ่านมา ไม่เคยเห็นข่าวดี ข่าวเชิงบวก ผลงานรัฐบาลที่เป็นที่ชื่นชอบประชาชนเลย รัฐบาลเผชิญสารพัดปัญหา เลยเป็นที่มาของแฮชแท็กในทวิตเตอร์เบื่อนายกฯ รัฐบาลเฮงซวย เพราะเป็นยุคที่ประชาชนต้องดูแลตัวเอง รัฐบาลไม่สามารถแก้ไขปัญหาต่างๆได้ เสียง ประชาชนที่สะท้อนออกมาตรงกับความรู้สึกพรรคเพื่อไทย ถ้า พล.อ.ประยุทธ์เหนื่อยก็พัก ไม่มีศักยภาพก็ลาออก เพื่อให้โหวตเลือกนายกฯมาใหม่ ถ้าฝืนความรู้สึกประชาชน ปัญหาจะจบอย่างไร สารพัดปัญหาที่รุมเร้าเป็นการวัดฝีมือว่ารัฐบาลไม่มีฝีมือบี้นายกฯไขก๊อกเซ่น ส.ส.กดบัตรแทนนายชุมสาย ศรียาภัย รองโฆษกพรรคเพื่อไทย กล่าวถึงการกดบัตรแทนกันของ ส.ส.ว่า ประธานรัฐสภาฯได้มีความเห็นว่าการเสียบบัตรแทนกัน ทำไม่ได้ มีโทษตามกฎหมาย จากประเด็นนี้อาจทำให้ร่าง พ.ร.บ.งบประมาณต้องตกไป ตามที่ศาล รัฐธรรมนูญได้เคยวินิจฉัยเอาไว้ ซึ่งถือเป็นบรรทัดฐาน โดยนายกฯต้องพิจารณายุบสภาฯหรือลาออก ถ้าฝืนมากไป จะมีแรงเสียดทานมากไปในทุกมิตินายการุณ โหสกุล ส.ส.กรุงเทพมหานคร พรรคเพื่อไทย กล่าวว่า ร่าง พ.ร.บ.งบฯปี 63 ฝ่ายค้านร่วมมือตลอด แต่ปัญหาที่เกิดขึ้นจากฝ่ายรัฐบาลไม่เกรงกลัวการทำผิดกฎหมาย จะมีผู้มีอำนาจหาทางออกให้ทุกเรื่อง การกดบัตรแทนกันกฎหมายบอกว่าต้องลงมติด้วยตัวเอง ผิดคือผิด จะพยายามหาทางออกช่วยคนกดบัตรแทนกันฟังไม่ขึ้น ส.ส.รัฐบาลตั้งใจทำผิดกฎหมาย เชื่อว่า พล.อ.ประยุทธ์จะเตรียมใช้อำนาจพิเศษออก พ.ร.ก.เบิกจ่ายงบฯขัดหลักประชาธิปไตย เน้นหลักกู ทั้งที่ พล.อ.ประยุทธ์พูดเสมอว่าทุกคนต้องเคารพกฎหมาย แต่ ส.ส.ฝ่ายรัฐบาลไม่เคารพกฎหมาย ปัญหาการเบิกจ่ายงบฯปี 63 เกิดจากฝ่ายรัฐบาลจงใจให้ล่าช้า ส่งผลให้เกิดวิกฤติอย่างแท้จริงเลื่อนเคาะอีกอ้างข้อมูลล้นทะลักเมื่อเวลา 17.30 น. พล.ท.ภราดร พัฒนถาบุตร เลขานุการคณะกรรมการกิจการพิเศษพรรคเพื่อไทย เปิดเผยว่า การยื่นญัตติอภิปรายไม่ไว้วางใจรัฐบาล ยังยึดวันที่ 29 ม.ค. เป็นหลัก คณะทำงานยกร่างญัตติกำลังขัดเกลาเนื้อหาที่ต้องบรรยายรัฐมนตรีที่ถูกยื่นอภิปราย นอกจาก 5 คนหลักรัฐมนตรีที่จะถูกอภิปรายเพิ่มพรรคเพื่อไทยยังต้องหารือกับพรรคร่วมฝ่ายค้านอีกครั้ง คงจะได้ข้อสรุปแน่นอนช่วงเย็นวันที่ 28 ม.ค.ที่ยังสรุปไม่ได้เนื่องจากมีข้อมูลหลั่งไหลเข้ามามาก ต้องร่วมกันพิจารณาว่าข้อมูลที่ได้มาลงลึกไปถึงตัวรัฐมนตรีคนนั้นๆจริงหรือไม่ เรากำลังจัดสรรผู้อภิปรายที่พรรคฝ่ายค้านแต่ละพรรคแสดงเจตจำนงเข้ามามาก แค่พรรคเพื่อไทย อนาคตใหม่ มีผู้ขออภิปรายพรรคละไม่ต่ำกว่า 10 คน ภายในสัปดาห์นี้พรรคร่วมฝ่ายค้านจะยื่นญัตติอภิปรายไม่ไว้วางใจได้แน่นอนตื่นเต้นข้อมูลใหม่ยังไม่ตัดชื่อ บิ๊กป้อมขณะที่นายสุทิน คลังแสง ประธานวิปฝ่ายค้าน กล่าวหลังประชุมวิปฝ่ายค้านว่า ขณะนี้วิปฝ่ายค้านยังไม่เคาะรายชื่อรัฐมนตรีที่จะถูกอภิปราย เพราะตื่นเต้นกับข้อมูลใหม่ๆที่หลั่งไหลเข้ามาต่อเนื่อง เราจะรอจนข้อมูลพร้อมสมบูรณ์ที่สุดถ้าเย็นวันที่ 29 ม.ค.ก็ยื่นวันนั้นเลย แต่อย่างไรไม่เกินสัปดาห์นี้ ส่วนชื่อ พล.อ.ประวิตร วงษ์สุวรรณ รองนายกฯ และ รมว.คนอื่นที่อยู่ในข่ายจะถูกอภิปรายเพิ่มยังไม่ได้ตัดทิ้งถกเครียดเทคนิคหวั่นเจอรุมประท้วงผู้สื่อข่าวรายงานจากที่ประชุมวิปฝ่ายค้านว่า ประเด็นที่วิปฝ่ายค้านหารือกันหนักคือข้อบังคับการอภิปรายไม่ไว้วางใจ ที่มีทั้งแบบยื่นญัตติทั้งคณะกับยื่นญัตติเป็นรายบุคคล เดิมที่ฝ่ายค้านจะยื่นเป็นรายบุคคล แต่ข้อบังคับการยื่นอภิปรายฯ หากยื่นเป็นรายบุคคลต้องอภิปรายจบเป็นคนๆไป แต่การบริหารงานของรัฐบาลนี้เชื่อมโยงกัน ทำให้การเขียนญัตติอภิปรายทำได้ยาก ต้องมาคิดว่าคนใดเชื่อมโยงกับ ใครบ้าง เพื่อจัดเรียงลำดับการร้อยเรียงอภิปรายให้ดี ป้องกันการประท้วงของพรรคร่วมรัฐบาล ทำให้แต่ละพรรคขอเอาประเด็นนี้ไปหารือกันอีกครั้งก่อนนำข้อสรุปมาประชุมในวันที่ 28 ม.ค.ต่อ เรื่องนี้ถือเป็นเทคนิคการอภิปรายไม่ไว้วางใจที่ต้องทำให้เรียบร้อยก่อนบิ๊กตู่ คืนถิ่นโรงเรียนเตรียมทหารเมื่อเวลา 09.00 น. ที่โรงเรียนเตรียมทหาร จ.นครนายก พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรีและ รมว.กลาโหม เป็นประธานงานเกียรติยศจักรดาวปี 2563 พร้อม พล.อ.ประวิตร วงษ์สุวรรณ รองนายกฯ พล.อ.อนุพงษ์ เผ่าจินดา รมว. มหาดไทย ร.อ.ธรรมนัส พรหมเผ่า รมช.เกษตรฯ ผบ.ทหารสูงสุด ผบ.เหล่าทัพ ผบ.ตร.ร่วมงานด้วย โดย พล.อ.ประยุทธ์ รับความเคารพจากกองทหารเกียรติยศก่อนวางพานพุ่มสักการะพระบรมราชานุสาวรีย์รัชกาลที่ 5 กล่าวนำคำปฏิญาณศิษย์เก่า วางพวงมาลาอนุสรณ์สถานโรงเรียนเตรียมทหาร แล้วเดินทักทายรุ่นพี่ที่ตั้งแถวกลางสนาม จากนั้นมีพิธีแสดงมุทิตาจิต อดีต ผบ.และครูอาจารย์ และมอบรางวัลเกียรติยศจักรดาวสดุดี ให้ศิษย์เก่าเตรียมทหารที่เสียสละชีวิตจากการปฏิบัติหน้าที่ราชการ ได้แก่ น.ต.ณฤพล เลิศกุศล ครูการบิน แอล-39 ที่สละชีวิตให้ลูกศิษย์รอด และมอบรางวัลเกียรติยศจักรดาวโดยตำแหน่ง ให้แก่ พล.อ.อ.มานัต วงษ์วาทย์ ผบ.ทอ.เป็นต้นโอดเร่งแก้สารพัดปัญหาเสียงด่าตรึมพล.อ.ประยุทธ์กล่าวว่า อบอุ่นทุกครั้งที่ได้กลับมาเยือน ขณะนี้เรามีปัญหาสำคัญต้องเร่งแก้ไข โดยเฉพาะฝุ่นละอองขนาดเล็ก ไข้หวัดโคโรนา หลายคนเสนอแนวทางปฏิบัติที่เป็นประโยชน์ แต่ต้องดูว่าทำอะไรได้บ้าง จะเอาเครื่องบินเข้าไปได้หรือไม่ขึ้นอยู่กับประเทศปลายทางเป็นผู้อนุมัติ แต่รัฐบาลเตรียมการมาแล้วติดต่อประสานงานกับต่างประเทศ ปัญหาเศรษฐกิจเราพยายามแก้ไขตามหลักการเศรษฐศาสตร์โลก ปัญหาการเมืองเกี่ยวกับงบประมาณทุกคนต้องเคารพกฎหมาย ปัญหาบริหารจัดการน้ำ การสร้างเขื่อนหรือทำอะไรในพื้นที่ต้องประชาพิจารณ์ ทุกเรื่องไม่สามารถทำได้วันเดียว หลายอย่างต้องใช้เวลา 5-20 ปีตามยุทธศาสตร์ชาติและแผนการปฏิรูป หลายอย่างจะทำตามใจไม่ได้ ต้องมีหลักคิด ข้อมูลและเหตุผล ยอมรับว่าที่ผ่านมามีกระแสวิพากษ์วิจารณ์แต่พยายามเดินหน้าต่อไป ไม่เคยทำให้สถาบันเสื่อมเสีย ยืนยันจะไม่มีการเอื้อประโยชน์ให้ใครคนใดคนหนึ่ง ต้องการสร้างหลักคิดที่ดีและถูกต้อง สิ่งใดไม่ถูกกฎหมายหรือไม่โปร่งใส มีการทุจริตต้องตรวจสอบและลงโทษ จากนั้นนายกฯลงมาทักทายศิษย์เก่าและศิษย์ปัจจุบันด้านล่างเวที ร่วมร้องเพลง ศรัทธา ให้กำลังใจนายกฯที่ยิ้มแย้มโบกมือตอบอ้างช่วงเปลี่ยนผ่านโจมตีกันมีแต่ถอยหลังต่อมาเวลา 14.00 น. ที่สภาอุตสาหกรรมแห่งประเทศไทย อาคารปฏิบัติการเทคโนโลยีเชิงสร้างสรรค์ มหาวิทยาลัยเทคโนโลยีราชมงคลกรุงเทพ พล.อ.ประยุทธ์ เป็นประธานในพิธีเปิดสภาอุตสาหกรรมแห่งประเทศไทยแห่งใหม่ มีเอกอัครราชทูต ผู้บริหารภาครัฐและเอกชน เข้าร่วมยินดี นายกฯยังได้กล่าวปาฐกถาพิเศษตอนหนึ่งว่า วันนี้เราต้องการคนรุ่นใหม่เข้ามาพัฒนาประเทศ โลกที่กำลังเติบโตทางเทคโนโลยี เกิดทั้งวิกฤติและโอกาสมาตลอด ต้องคิดหาทางปฏิบัติที่เหมาะสมและเป็นไปได้ ตนไม่ประกาศว่ารู้เก่งกว่าใคร แต่จะแสวงหาแนวทางขับเคลื่อนสิ่งต่างๆ ให้ได้ผล แต่สิ่งสำคัญต้องทำให้ประชาชนเข้าใจไม่อย่างนั้นขัดแย้งกันตลอด อย่าลืมว่าช่วงนี้เป็นช่วงเปลี่ยนผ่านประเทศไปสู่ความมั่นคง มั่งคั่งและยั่งยืน ไม่ใช่เรื่องง่าย หากโจมตีกันไปมีแต่ถอยหลัง ต้องสามัคคีรับมือสถานการณ์ต่างๆอ้อนร่วมแรงมา 5 ปีอย่าเพิ่งทิ้งกันนายกฯกล่าวอีกว่า ทุกอย่างมีกฎหมายและมาตรการต่างๆ แต่หลายอย่างบังคับด้วยอำนาจทางกฎหมายและอำนาจบริหารไม่ได้ ทุกอย่างถูกตรวจสอบอันเดียวกันหมด ทั้งองค์กรของรัฐและองค์กรอิสระรัฐบาลถูกตรวจสอบมาโดยตลอด นายกฯโดนแบบนี้ ชี้แจงไม่เว้นแต่ละวันชี้แจงได้จบ ถ้าไม่ได้ก็เข้าสู่กระบวนการยุติธรรม วันนี้เราร่วมชะตากรรมกันมาเยอะแล้ว มีทั้งชื่นชม ถูกบ่น ถูกว่าแล้วเราจะทิ้งกันไปได้อย่างไร หรือจะทิ้งตนก็ตามใจท่าน ไม่มีปัญหาอะไรทั้งสิ้น เพราะไม่ยึดติดอะไรทั้งนั้น เพียงแต่อยากให้สำเร็จ ในเมื่อเราลงทุนลงแรงกันมา 5 ปีแล้วเราจะสู้กันต่อไหม ศรัทธาต่อรัฐบาล และ ครม.ต่อไหม ผู้สื่อข่าวรายงานว่า ขณะนี้ผู้ร่วมงานภายในห้องตอบกลับว่า สู้ และช่วงท้ายนายกฯกล่าวว่า ฝากถึงสื่อด้วยวันนี้สาระทั้งหมดเป็นเรื่องพัฒนาอุตสาหกรรม ไม่ใช่เรื่องการเมือง อย่ามาถามอีกความขัดแย้ง ถามได้ทุกเรื่องถามทุกที่ พอพูดผิดตรงไหนก็เอาแล้ว นั่นแหละทำลายประเทศจารุวรรณ ยื่นร้องสอดคดียุบ อนค.เมื่อเวลา 10.00 น. ที่ศาลรัฐธรรมนูญ น.ส.จารุวรรณ ศรัณย์เกตุ ส.ส.บัญชีรายชื่อ และกรรมการบริหารพรรคอนาคตใหม่ พร้อมนายอนาวิล รัตนสถาพร ส.ส.ปทุมธานี เขต 3 พรรคอนาคตใหม่ เดินทางมายื่นหนังสือต่อศาลรัฐธรรมนูญ เกี่ยวเนื่องกับคดีที่ กกต.ส่งคำร้องให้ยุบพรรคอนาคตใหม่ จากกรณีกู้เงินนายธนาธร จึงรุ่งเรืองกิจ โดย น.ส.จารุวรรณ กล่าวว่า เนื่องจากตนเป็นกรรมการบริหารพรรคอนาคตใหม่ ถือว่าเป็นผู้ถูกร้องคดีนี้เช่นกัน จึงขอใช้สิทธิในฐานะผู้ถูกร้อง โดยขอร้องสอดเข้ามาเป็นผู้ถูกร้องที่ 2 เพื่อขอใช้สิทธิยื่นคำชี้แจงและแก้ข้อกล่าวหาทั้งแสดงพยานหลักฐานต่างๆเพื่อใช้สิทธิในการต่อสู้คดีอย่างเต็มที่และเพื่อความเป็นธรรมขอขยายเวลาส่งเอกสารแจงอีก 30 วันน.ส.จารุวรรณกล่าวอีกว่า รวมทั้งขอขยายเวลาการยื่นเอกสารชี้แจงต่อศาลออกไปอีก 30 วัน เนื่องจากได้ไปยื่นขอคัดสำนวนคำวินิจฉัยของคณะกรรมการไต่สวน สืบสวน ตามข้อกล่าวหา เพื่อใช้สิทธิตามกฎหมายในการที่จะได้เป็นข้อมูลในการต่อสู่คดีตามกระบวนการของกฎหมายที่ถูกต้อง ซึ่งได้ทวงถามกับ กกต.3 ครั้ง แต่เลขาธิการ กกต.ไม่ให้ตามร้องขอ จึงขอใช้สิทธิอุทธรณ์คำสั่งต่อสำนักงานคณะกรรมการข้อมูลข่าวสารของราชการ สำนักงานปลัดสำนักนายกรัฐมนตรี ทำเนียบรัฐบาล เมื่อวันที่ 22 ม.ค. เพื่อจะได้เป็นข้อมูลในการต่อสู้คดีตามกระบวนการกฎหมายอย่างถูกต้อง ยุติธรรม ปราศจากอคติใดๆ แต่ก็ยังไม่ได้รับเอกสารใดๆตามที่ร้องขอ23 ก.พ. เลือกซ่อมกำแพงเพชรผู้สื่อข่าวรายงานว่าสำนักงาน กกต.เผยแพร่เอกสารข่าวการเลือกตั้ง ส.ส.จังหวัดกำแพงเพชรเขตเลือกตั้งที่ 2 แทนตำแหน่งที่ว่าง ตามที่ได้มี พ.ร.ฎ.ให้มีการเลือกตั้ง ส.ส.กำแพงเพชรเขต 2 นั้น กกต.ได้ออกประกาศ กกต. เรื่องกำหนดวันเลือกตั้งและวันรับสมัครเลือกตั้ง ส.ส.กำแพงเพชรเขต 2 ในวันอาทิตย์ที่ 23 ก.พ.2563 เวลา 08.00-17.00 น. เปิดวันรับสมัครเลือกตั้งวันพุธที่ 29 ม.ค.-2 ก.พ.เวลา 08.30-16.30 น. ณ สถานที่ผู้อำนวยการเลือกตั้งประจำเขตเลือกตั้งที่ 2 กำหนด สำหรับการหาเสียงให้กระทำได้ตั้งแต่วันที่ตำแหน่งว่างลงจนถึงวันก่อนวันเลือกตั้ง ตาม พ.ร.บ.ประกอบรัฐธรรมนูญว่าด้วยการเลือกตั้ง ส.ส. พ.ศ.2561 มาตรา 68 (2)'''

#### Functions

*Sea*: **TF-IDF & TextRank weight is *Dictionary*** เลยขอเขียน condition เพิ่มใน ``highlightedText`` นะค้าบ

In [41]:
from collections import OrderedDict
import numpy as np
import bisect
from sklearn.feature_extraction.text import TfidfVectorizer

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai

# Highlight & export function
def highlightedText(tokens, attention, encoder1: int=11, encoder2: int=8, pad='', max_alpha=0.8, save_html=False) -> list:
    '''
    https://adataanalyst.com/machine-learning/highlight-text-using-weights/
    '''
    highlighted_text = []
    for i, word in enumerate(tokens):
        # Update ====================
        word = str(word)
        # ===========================

        if '▁' in word: word = word.replace('▁', ' ') # change space '▁' to ' '

        # TF-IDF & TextRank weight is dictionary =================================================================================
        if isinstance(attention, dict):
            
            if word in attention.keys():
                weight = attention[word] / max(attention.values()) # normalize
            else:
                weight = 0.0

        # ========================================================================================================================

        else: # else for Attention in tranformer which has Multi-dimension
            df_sumcol = pd.DataFrame( attention[encoder1][0][encoder2] ).sum(axis=0) # reduce dim(N, N) to dim(N, 1) by sum column
            df_sumcol /= df_sumcol.abs().max() # normalize

            weight = df_sumcol[i]
        # ========================================================================================================================
        
        highlighted_text.append('<span style="background-color:rgba(251,155,59,' + str(weight * max_alpha) + ');">' + html_escape(word) + '</span>')
        # highlighted_text.append('' + html_escape(word) + '')
        
    highlighted_text = pad.join(highlighted_text)

    display(HTML(highlighted_text))

    if save_html:
        html = HTML(highlighted_text).data
        # file name ex. attention_0_1.html
        with open(f'attention_{encoder1}_{encoder2}.html', 'w') as f:
            f.write(html)

    return highlighted_text

class Base:
    def __init__(self):
        self.min_length = 50    # max_length = 2*self.min_lenght

        self.feedback_code = {
            100: "failed to generate output",
            200: "success",
            300: "failed to decode output",
            400: "failed to hightlight input text",
            500: "input summarized length is underexpected",
            600: "inpit summarized length is overexpected "       # (not set yet)
        }
    def summarize(self):
        return 
        
    def get_longer(self):
        self.min_length += 50
        return self.summarize()
    
    def get_shorter(self):
        if self.min_length <= 0:
            return 500 , self.feedback_code[500], -1
        self.min_length -= 50
        return self.summarize()
    
    def reset_min_length(self):
        self.min_length = 50


class TFIDF(Base):
    def __init__(self, method='max'):
        super().__init__()  # Call the parent class constructor

        self.segmenter = sentence_segment
        self.tokenizer = spacy_thai.load()
        self.tokens = None

        self.vectorizer = TfidfVectorizer(tokenizer=self.tokenizer)
        self.method = method # {avg, max}

        self.min_length = 50    # max_length = 2*self.min_lenght
    
    def tokenize(self, inputs):
        self.inputs = inputs
        self.prev_inputs = None
        sentences = self.segmenter(inputs)
        # convert type`thai_segmenter.sentence.sentence` to `str`
        self.sentences = [str(sentence) for sentence in sentences]

        self.tokens = self.tokenizer(inputs)
        return self.tokens

    def summarize(self):
        ''''
            feedback code:
                100: failed to generate output
                200: success
                300: failed to decode output
                400: failed to hightlighted input text
                500: input summarized length is underexpected
                600: inpit summarized length is overexpected        (not set yet)
        '''
        # --------------------------------------Calculation part--------------------------------------#
        try:
            if self.prev_inputs == None:
                
                mat = self.vectorizer.fit_transform(self.sentences)
                # tokens = self.tokenizer( ' '.join(self.sentences) )

                # print(type(mat), mat.toarray(), mat.toarray().shape, np.average(mat.toarray(),axis=1).shape)

                sentence_weight = np.average(mat.toarray(),axis=1)
                word_weight     = mat.sum(axis=0).A1

                sentence_weight_norm = (sentence_weight-np.min(sentence_weight))/(np.max(sentence_weight)-np.min(sentence_weight))
                word_weight_norm     = (word_weight-np.min(word_weight))/(np.max(word_weight)-np.min(word_weight))

                self.node_weight = dict()
                for word, word_id in self.vectorizer.vocabulary_.items(): # replace word_id -> weight
                    if str(word) not in self.node_weight.keys(): 
                        self.node_weight[str(word)] = word_weight_norm[word_id]
                    else: # is already have?
                        self.node_weight[str(word)] = sum([self.node_weight[str(word)], word_weight_norm[word_id]]) / 2

        # --------------------------------------Getting sentence part--------------------------------------#

                self.sentences_weight = [] #sentence_weight
                self.sentences_rank = [] #np.argsort(sentence_weight, axis=0)[::-1]
                self.sentences_len = [] #[len(self.tokenizer(str(sentence))) for sentence in self.sentences]

                # Add-on for more sentence weight calculation
                # 
                for sentence in self.sentences:
                    if self.method == 'avg':
                    # Avg score method --------------------------------- 
                        score = []
                        tokens = self.tokenizer(str(sentence))
                        for token in tokens:
                            score.append(self.node_weight[ str(token) ] if str(token) in self.node_weight.keys() else 0.0)
                        self.sentences_weight.append( np.average(np.asarray(score)) )

                    elif self.method == 'max':
                    # Max score method --------------------------------- 
                        score = 0.0
                        tokens = self.tokenizer(str(sentence))
                        for token in tokens:
                            score += (self.node_weight[ str(token) ] if str(token) in self.node_weight.keys() else 0.0)
                        self.sentences_weight.append( score )
                    
                    else: print("Invalid method!")
                    
                    self.sentences_len.append( len(tokens) )
                self.sentences_rank = np.argsort(np.asarray(self.sentences_weight), axis=0)[::-1]

                self.prev_inputs = self.inputs
        except:
            return 100, self.feedback_code[100], -1

        # --------------------------------------Generating output part--------------------------------------#
        try:
            output = []
            current_len = 0; current_rank = []
            for rank in self.sentences_rank:
                to_insert = bisect.bisect_left(current_rank, rank)
                output.insert(
                    to_insert,
                    str(self.sentences[rank]),
                )
                current_rank.insert(to_insert, rank)
                current_len += self.sentences_len[rank]

                # print(current_rank); print(current_len); print(output)
                
                if current_len >= self.min_length: break
            output = ''.join(output)
        except:
            return 300, self.feedback_code[300], -1

        try:
            highlighted_text = highlightedText( list(self.tokens), self.node_weight)
        except:
            return 400, self.feedback_code[400], -1
        return 200, output, highlighted_text


class TextRank(Base):
    def __init__(self, method='max'):
        super().__init__()  # Call the parent class constructor
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

        self.method = method # {avg, max}

        self.segmenter = sentence_segment
        self.tokenizer = spacy_thai.load()
        self.tokens = None

        self.min_length = 50    # max_length = 2*self.min_lenght
    
    def tokenize(self, inputs):
        self.inputs = inputs
        self.prev_inputs = None
        self.tokens = self.tokenizer(inputs)
        return self.tokens

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    def summarize(self, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        ''''
            feedback code:
                100: failed to generate output
                200: success
                300: failed to decode output
                400: failed to hightlighted input text
                500: input summarized length is underexpected
                600: inpit summarized length is overexpected        (not set yet)
        '''
        # --------------------------------------Calculation part--------------------------------------#
        try:
            if self.prev_inputs == None:
                # Filter sentences
                sentences = self.sentence_segment(self.tokens, candidate_pos, lower) # list of list of words
                # Build vocabulary
                vocab = self.get_vocab(sentences)
                # Get token_pairs from windows
                token_pairs = self.get_token_pairs(window_size, sentences)
                # Get normalized matrix
                g = self.get_matrix(vocab, token_pairs)
                # Initionlization for weight(pagerank value)
                pr = np.array([1] * len(vocab))
                # Iteration
                previous_pr = 0
                for epoch in range(self.steps):
                    pr = (1-self.d) + self.d * np.dot(g, pr)
                    if abs(previous_pr - sum(pr))  < self.min_diff:
                        break
                    else:
                        previous_pr = sum(pr)
                # Get weight for each node
                node_weight = dict()
                for word, index in vocab.items():
                    node_weight[word] = pr[index]
                self.node_weight = node_weight
        # --------------------------------------Getting sentence part--------------------------------------#
                self.sentences = self.segmenter( self.inputs )
                self.sentences_weight = []
                self.sentences_rank = []
                self.sentences_len = []

                for sentence in self.sentences:
                    if self.method == 'avg':
                    # Avg score method --------------------------------- 
                        score = []
                        tokens = self.tokenizer(str(sentence))
                        for token in tokens:
                            score.append(self.node_weight[ str(token) ] if str(token) in self.node_weight.keys() else 0.0)
                        self.sentences_weight.append( np.average(np.asarray(score)) )

                    elif self.method == 'max':
                    # Max score method --------------------------------- 
                        score = 0.0
                        tokens = self.tokenizer(str(sentence))
                        for token in tokens:
                            score += (self.node_weight[ str(token) ] if str(token) in self.node_weight.keys() else 0.0)
                        self.sentences_weight.append( score )
                    
                    else: print("Invalid method!")
                    
                    self.sentences_len.append( len(tokens) )
                self.sentences_rank = np.argsort(np.asarray(self.sentences_weight), axis=0)[::-1]

                self.prev_inputs = self.inputs
        except:
            return 100, self.feedback_code[100], -1

        # --------------------------------------Generating output part--------------------------------------#
        try:
            output = []
            current_len = 0; current_rank = []
            for rank in self.sentences_rank:
                to_insert = bisect.bisect_left(current_rank, rank)
                output.insert(
                    to_insert,
                    str(self.sentences[rank]),
                )
                current_rank.insert(to_insert, rank)
                current_len += self.sentences_len[rank]

                # print(current_rank); print(current_len); print(output)
                
                if current_len >= self.min_length: break
            output = ''.join(output)
        except:
            return 300, self.feedback_code[300], -1

        try:
            highlighted_text = highlightedText( list(self.tokens), self.node_weight)
        except:
            return 400, self.feedback_code[400], -1
        return 200, output, highlighted_text

#### **Usage**

In [38]:
# TF-IDF
tfidf = TFIDF()
#       TFIDF( method='avg' ) (Optional)
tfidf.tokenize(paragraph)
status, output, highlighted_text = tfidf.summarize()

print(status, output)

print("tokens:          ", len(tfidf.tokens))
print("word_weight:     ", len(tfidf.node_weight))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


200 นายสมพงษ์ กล่าวในที่ประชุมว่า พรรคร่วมฝ่ายค้านที่จะยื่นญัตติอภิปรายไม่ไว้วางใจรัฐบาล เดิมกำหนดไว้วันที่ 29 ม.ค. แต่ถ้าหากจะมีการเลื่อนออกไปจากเดิม 1-2 วัน ไม่น่ามีปัญหา และคงจะไม่กระทบกับกรอบวันของการอภิปรายไม่ไว้วางใจฉะให้ยับ รบ.ไร้น้ำยาบริหารประเทศนายภูมิธรรมกล่าวว่า การอภิปรายครั้งนี้พรรคร่วมฝ่ายค้านตั้งใจให้ประชาชนเข้ามามีส่วนร่วมตรวจสอบรัฐบาลด้วย ไม่ได้หวังพึ่งมือในสภาฯเพียงอย่างเดียว รัฐบาลแม้จะเป็นเสียงข้างมาก แต่อยู่ในภาวะเสียงปริ่มน้ำ ฝ่ายค้านจะชี้ให้ประชาชนได้ประจักษ์ว่า การบริหารงานที่ผ่านมาของรัฐบาล ก่อให้เกิดประสิทธิภาพหรือไม่ ทั้ง 7 พรรคร่วมฝ่ายค้าน มีความพร้อมมีความเข้าใจร่วมกัน
tokens:           4607
word_weight:      1350


In [ ]:
# status, output, highlighted_text = textrank.get_longer()
# status, output, highlighted_text = textrank.get_shorter()
# print(status, output)

*Sea*: **TF-IDF** จะไม่ค่อยเวิร์คกับประโยค(สั้น ๆ) และ paragraph สั้น ๆ(จำนวนประโยคน้อย) **เหมาะกับ paragraph ยาว ๆ มากกว่า**

In [42]:
# TextRank
textrank = TextRank()
#          TextRank( method='avg' ) (Optional)
textrank.tokenize(paragraph)
status, output, highlighted_text = textrank.summarize( window_size=4 )

print(status, output)

print("tokens:          ", len(textrank.tokens))
print("word_weight:     ", len(textrank.node_weight))

200 จึงทำให้ไม่สามารถเสียบบัตรแทนกันได้ และเชื่อว่าการลงโทษ ส.ส.ในครั้งนี้จะเป็นบทเรียนให้กับ ส.ส.ในสภาฯสมพงษ์ ชี้เลื่อนยื่นญัตติไม่กระทบต่อมาเวลา 11.30 น. ที่พรรคเพื่อไทย นายสมพงษ์ อมรวิวัฒน์ หัวหน้าพรรคเพื่อไทยและผู้นำฝ่ายค้าน นายภูมิธรรม เวชยชัย ที่ปรึกษาผู้นำฝ่ายค้าน น.อ.อนุดิษฐ์ นาครทรรพ เลขาธิการพรรคเพื่อไทย นายสุทิน คลังแสง ประธานวิปฝ่ายค้าน นายธนาธร จึงรุ่งเรืองกิจ หัวหน้าพรรคอนาคตใหม่ นายวันมูหะมัดนอร์ มะทา หัวหน้าพรรคประชาชาติ นพ.เรวัต วิศรุตเวช ส.ส.บัญชีรายชื่อพรรคเสรีรวมไทย นายสงคราม กิจเลิศไพโรจน์ หัวหน้าพรรคเพื่อชาติ นายนิคม บุญวิเศษ หัวหน้าพรรคพลังปวงชนไทย ร่วมประชุมเพื่อกำหนดกรอบประเด็นและรัฐมนตรีที่จะถูกยื่นญัตติอภิปรายไม่ไว้วางใจ
tokens:           4607
word_weight:      726


In [43]:
# TextRank
textrank = TextRank( method='avg' )

textrank.tokenize(paragraph)
status, output, highlighted_text = textrank.summarize( window_size=4 )

print(status, output)

print("tokens:          ", len(textrank.tokens))
print("word_weight:     ", len(textrank.node_weight))

200 นายศรีสุวรรณ จรรยา เลขาธิการสมาคมองค์การพิทักษ์รัฐธรรมนูญไทย ยื่นคำร้องต่อ ป.ป.ช.ให้เอาผิด 3 ส.ส.พรรคภูมิใจไทย และ 1 ส.ส.พรรคพลังประชารัฐที่กดบัตรแทนกันจะได้ไม่อับอายต่อประชาคมโลกอีกต่อไป นายเทพไทกล่าวฝ่ายค้านคุยข้อมูลเชือดไหลเข้าไม่หยุดเมื่อเวลา 10.00 น. ที่พรรคเพื่อไทย นายสุทิน คลังแสง ประธานวิปฝ่ายค้าน กล่าวถึงการยื่นญัตติอภิปรายไม่ไว้วางใจรัฐบาล พล.อ.ประยุทธ์ จันทร์โอชา ว่า วิปฝ่ายค้านพร้อมเต็มที่5 คนหลักรัฐมนตรีที่จะถูกอภิปรายเพิ่มพรรคเพื่อไทยยังต้องหารือกับพรรคร่วมฝ่ายค้านอีกครั้ง
tokens:           4607
word_weight:      726


In [ ]:
# status, output, highlighted_text = textrank.get_longer()
# status, output, highlighted_text = textrank.get_shorter()
# print(status, output)

*Sea*: **TextRank** จะเน้น highlight **keywords** และจะดึงประโยคที่มี keywords มากที่สุด

# **Inference** (*Old*)

In [ ]:
!pip install thai-segmenter spacy_thai

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import html
from IPython.core.display import display, HTML
from collections import OrderedDict

# Prevent special characters like & and < to cause the browser to display something other than what you intended.
def html_escape(text):
    return html.escape(text)

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai

In [ ]:
paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

tokenizer = spacy_thai.load()

sentences = sentence_segment(paragraph)

# convert type`thai_segmenter.sentence.sentence` to `str`
sentences = [str(tokenizer(str(sentence))) for sentence in sentences]


#### Functions

*Sea*: **TF-IDF & TextRank weight is *Dictionary*** เลยขอเขียน condition เพิ่มใน ``highlightedText`` นะค้าบ

In [ ]:
def getRankTFIDF(sentences, tokenizer, ngram_range=(1,1)):

    vectorizer = TfidfVectorizer( 
        tokenizer=tokenizer,
        ngram_range=ngram_range
    )
    mat = vectorizer.fit_transform(sentences)
    tokens = tokenizer( ' '.join(sentences) )
    # tokens = vectorizer.get_feature_names_out()

    # print(mat.shape)
    # print("tokens:", len(tokens), tokens)

    sentence_weight = mat.sum(axis=1).A1
    word_weight     = mat.sum(axis=0).A1

    sentence_weight_norm = (sentence_weight-np.min(sentence_weight))/(np.max(sentence_weight)-np.min(sentence_weight))
    word_weight_norm     = (word_weight-np.min(word_weight))/(np.max(word_weight)-np.min(word_weight))

    word_weight_dict = dict()
    for word, word_id in vectorizer.vocabulary_.items(): # replace word_id -> weight
        if str(word) not in word_weight_dict.keys(): 
            word_weight_dict[str(word)] = word_weight_norm[word_id]
        else: # is already have?
            word_weight_dict[str(word)] = sum([word_weight_dict[str(word)], word_weight_norm[word_id]]) / 2

    sentence_rank = np.argsort(sentence_weight, axis=0)[::-1]
    # word_rank     = np.argsort(word_weight, axis=0)[::-1]

    return sentence_rank, sentence_weight_norm, tokens, word_weight_dict

In [ ]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text
    
    https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py"""
    
    def __init__(self, tokenizer):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        self.tokenizer = tokenizer
        self.tokens = []
    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        # for word in STOP_WORDS.union(set(stopwords)):
        #     lexeme = nlp.vocab[word]
        #     lexeme.is_stop = True
        pass
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = self.tokenizer(text)
        self.tokens = doc
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
# Highlight & export function
def highlightedText(tokens, attention, encoder1: int=11, encoder2: int=8, pad='', max_alpha=0.8, save_html=False) -> list:
    '''
    https://adataanalyst.com/machine-learning/highlight-text-using-weights/
    '''
    highlighted_text = []
    for i, word in enumerate(tokens):
        # Update ====================
        word = str(word)
        # ===========================

        if '▁' in word: word = word.replace('▁', ' ') # change space '▁' to ' '

        # TF-IDF & TextRank weight is dictionary =================================================================================
        if isinstance(attention, dict):
            
            if word in attention.keys():
                weight = attention[word] / max(attention.values()) # normalize
            else:
                weight = 0.0

        # ========================================================================================================================

        else: # else for Attention in tranformer which has Multi-dimension
            df_sumcol = pd.DataFrame( attention[encoder1][0][encoder2] ).sum(axis=0) # reduce dim(N, N) to dim(N, 1) by sum column
            df_sumcol /= df_sumcol.abs().max() # normalize

            weight = df_sumcol[i]
        # ========================================================================================================================
        
        highlighted_text.append('<span style="background-color:rgba(251,155,59,' + str(weight * max_alpha) + ');">' + html_escape(word) + '</span>')
        # highlighted_text.append('' + html_escape(word) + '')
        
    highlighted_text = pad.join(highlighted_text)

    display(HTML(highlighted_text))

    if save_html:
        html = HTML(highlighted_text).data
        # file name ex. attention_0_1.html
        with open(f'attention_{encoder1}_{encoder2}.html', 'w') as f:
            f.write(html)

    return highlighted_text

#### **Usage**

In [ ]:
# TF-IDF
sentence_rank, sentence_weight, tokens, word_weight = getRankTFIDF(sentences, tokenizer)

print("sentence_rank:   ", sentence_rank)
print("sentence_weight: ", sentence_weight)
print("tokens:          ", len(tokens))
print("word_weight:     ", word_weight)

print()
_ = highlightedText( list(tokens), word_weight)

sentence_rank:    [2 0 6 4 5 3 1]
sentence_weight:  [0.83419727 0.         1.         0.42768911 0.62450999 0.57953056
 0.74772657]
tokens:           230
word_weight:      {'แนวโน้ม': 0.04188932728207886, 'ของ': 0.06903521283340266, 'การ': 0.04188932728207886, 'ใช้': 0.13465130795613228, 'หุ่นยนต์': 0.08128498115644184, 'ทาง': 0.055488625209436884, 'ด้าน': 0.06903521283340266, 'การผลิต': 0.10849017254830073, 'และ': 0.09895205368377233, 'งาน': 0.04188932728207886, 'บริการ': 0.08976012931746735, 'มี': 0.10325400663804088, 'จำนวน': 0.04188932728207886, 'สูง': 0.04188932728207886, 'ขึ้น': 0.13465130795613228, 'มากกว่า': 0.07421866844027028, '30': 0.04188932728207886, '%': 0.04188932728207886, 'ทุกปี': 0.04188932728207886, 'แต่': 0.04188932728207886, 'ประเทศ': 0.04188932728207886, 'ไทย': 0.010472331820519714, 'ยัง': 0.04188932728207886, 'ขาด': 0.04188932728207886, 'บุคลากร': 0.04188932728207886, 'วิทยาการ': 0.039780127478657314, 'ระบบ': 0.0688770819232259, 'อัตโนมัติ': 0.02094466364103943, 

*Sea*: **TF-IDF** จะไม่ค่อยเวิร์คกับประโยค(สั้น ๆ) และ paragraph สั้น ๆ(จำนวนประโยคน้อย) **เหมาะกับ paragraph ยาว ๆ มากกว่า**

In [ ]:
# TextRank
tr4w = TextRank4Keyword( tokenizer=tokenizer )
tr4w.analyze(paragraph, window_size=4, lower=False)

tokens      = tr4w.tokens
word_weight = tr4w.node_weight

print("tokens:          ", len(tokens))
print("word_weight:     ", len(word_weight))

print()
_ = highlightedText( list(tokens), word_weight)

tokens:           234
word_weight:      78



*Sea*: **TextRank** จะเน้น highlight **keywords**

---

# **Build the model**

### **Thai sentence segmentation**

💡 [thai-segmenter](https://pypi.org/project/thai-segmenter/)

In [ ]:
!pip install thai-segmenter
# https://pypi.org/project/thai-segmenter/

In [ ]:
from thai_segmenter import sentence_segment

In [ ]:
paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

sentences = sentence_segment(paragraph)

# convert type`thai_segmenter.sentence.sentence` to `str`
sentences = [str(sentence) for sentence in sentences]

for sentence in sentences:
    print('-', str(sentence))

- แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น
- ในปี 2538
- รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ
- ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้
- บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ
- มีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้
- 

## **TF-IDF** (old version)

In [ ]:
!pip install spacy_thai thai_segmenter # thai_tokenizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai
from thai_tokenizer import Tokenizer

In [ ]:
# def sentenceSplitter(paragraph, min_len=60):
#     paragraph = paragraph.replace("\n","")
#     sentences = paragraph.split(" ")

#     for i in range(len(sentences)):

#         while len(sentences[i]) < min_len and i>0:
#             # print(i, sentences[i], len(sentences))
#             sentences[i-1] += sentences.pop(i)

#             if i>=(len(sentences)): break

#         if i>=(len(sentences)): break
#     return sentences

In [ ]:
tokenizer = spacy_thai.load()
# tokenizer = Tokenizer() # thai-tokenizer

paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''
# old
# sentences = sentenceSplitter(paragraph, min_len=60)
# len(sentences), sentences

# new
sentences = sentence_segment(paragraph)

# convert type`thai_segmenter.sentence.sentence` to `str`
sentences = [str(tokenizer(str(sentence))) for sentence in sentences]
# sentences = [str(sentence) for sentence in sentences]

for sentence in sentences:
    print('-', str(sentence))

- แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น
- ในปี 2538
- รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ
- ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้
- บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ
- มีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้
- 

In [ ]:
def getRankTFIDF(sentences, tokenizer, ngram_range=(1,1)):

    vectorizer = TfidfVectorizer( 
        tokenizer=tokenizer,
        ngram_range=ngram_range
    )
    mat = vectorizer.fit_transform(sentences)
    print("tokens:", len(tokens:=vectorizer.get_feature_names_out()), tokens)
    print("tokens:", len(tokens:=list(vectorizer.vocabulary_)), tokens)

    # print(mat.shape)

    sentence_weight = mat.sum(axis=1).A1
    word_weight     = mat.sum(axis=0).A1

    sentence_weight_norm = (sentence_weight-np.min(sentence_weight))/(np.max(sentence_weight)-np.min(sentence_weight))
    word_weight_norm     = (word_weight-np.min(word_weight))/(np.max(word_weight)-np.min(word_weight))

    sentence_rank = np.argsort(sentence_weight, axis=0)[::-1]
    word_rank     = np.argsort(word_weight, axis=0)[::-1]

    return sentence_rank, sentence_weight_norm, tokens, word_rank, word_weight_norm

sentence_rank, sentence_weight, tokens, word_rank, word_weight = getRankTFIDF(sentences, tokenizer)

tokens: 230 [แนวโน้ม ใน รศ. ทุก บัณฑิต มี การนำ ปี ความร่วมมือ หลักสูตร ดร. 2538 ของ
 ฟี ของ ชิต โบ้ รศ. การ เหล่า เน้น เป็นที่ต้องการ ใช้ ที่ ดร. หลัก
 หุ่นยนต์ วัฒนา แข็งแกร่ง สยาม outcome-based เจริญ และ ทาง ฟี ของ กับ
 เสียง ด้าน โบ้ ภาคเอกชน ภาครัฐ มี การผลิต คติพจน์ ผู้อำนวยการ education
 และ และ ภาค และ ว่า ภาครัฐ ธุรกิจ งาน “ สถาบัน บริการ a ที่ อย่าง cradle
 อุตสาหกรรม บัณฑิต ฯ มี มาก มุ่งมั่น จำนวน of สามารถ นอกจากนี้ future
 ชั้นนำ สูง ใช้ เพื่อ ขึ้น สมรรถนะ leaders ฟี ทั้ง มากกว่า ยกระดับ โบ้
 กลุ่ม มี ที่ ผลงานวิจัย in ฟี 30 เรียน ปิโตรเลียม โบ้ % robotics ทุกปี
 เข้าสู่ ใน การพัฒนา / พัฒนา แต่ ” อาหาร สถาบัน ที่ ประเทศ และ มุ่ง
 เทคโนโลยี ให้บริการ ฮาร์ดดิสก์ ชั้นนำ ไทย สร้าง ยัง ระดับ ผู้นำ หุ่นยนต์
 ขาด วิชาการ บุคลากร การผลิต อาเซียน ด้าน ขึ้น วิทยาการ ด้าน ด้าน ไฟฟ้า
 ทาง ได้ หุ่นยนต์ ด้าน วิทยาการ หุ่นยนต์ การขนส่ง วิทยาการ เพื่อ หุ่นยนต์
 ของ การผลิต ธุรกิจ หุ่นยนต์ ไทย และ ฟี บริการ และ ระบบ โบ้ รวมทั้ง เพื่อ
 เป็น อัตโนมัติ การศึกษา สถาบัน พัฒนา สร้าง คุณภาพชีวิต 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
print("sentence_rank:   ", sentence_rank)
print("sentence_weight: ", sentence_weight.shape, sentence_weight)
print("word_weight:     ", word_weight.shape)

sentence_rank:    [2 0 6 4 5 3 1]
sentence_weight:  (7,) [0.83419727 0.         1.         0.42768911 0.62450999 0.57953056
 0.74772657]
word_weight:      (230,)


In [ ]:
word_rank, word_weight

In [ ]:
# Example: show top-3 sentences
print(np.asarray(sentences)[sentence_rank[:3]], '\n')

# Example: show top-10 keywords
for rnk in word_rank[:10]:
    print(tokens[rnk], '-', word_weight[rnk])

['รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ'
 'แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น'
 'การนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'] 

ของ - 1.0
การผลิต - 1.0
มี - 1.0
วิทยาการ - 0.2274132886301857
อัตโนมัติ - 0.2274132886301857
คติพจน์ - 0.2274132886301857
ตั้งขึ้น - 0.2274132886301857
cradle - 0.2274132886301857
ที่ - 0.2274132886301857
โบ้ - 0.2274132886301857


Old version

In [ ]:
# TF-IDF
sentence_rank, sentence_weight, tokens, word_rank, word_weight = getRankTFIDF(sentences, tokenizer)

print("sentence_rank:   ", sentence_rank)
print("sentence_weight: ", sentence_weight)
print("tokens:          ", tokens.shape)
print("word_rank:       ", word_rank.shape)
print("word_weight:     ", word_weight.shape)

print()
_ = highlightedText( list(tokens), word_weight)

sentence_rank:    [2 0 6 4 5 3 1]
sentence_weight:  [0.83419727 0.         1.         0.42768911 0.62450999 0.57953056
 0.74772657]
tokens:           (230,)
word_rank:        (230,)
word_weight:      (230,)



In [ ]:
# # Increse N-grams 
# sentence_rank, sentence_weight, tokens, word_rank, word_weight = getRankTFIDF(sentences, tokenizer, ngram_range=(1,3))

# # Example: show top-3 sentences
# print(np.asarray(sentences)[sentence_rank[:3]], '\n')

# # Example: show top-10 keywords
# for rnk in word_rank[:10]:
#     print(tokens[rnk], '-', word_weight[rnk])

## **TestRank**

💡 [@BrambleXu](https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py) |
[Medium](https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0)

In [ ]:
# !pip install pytextrank

In [ ]:
import spacy
# import pytextrank

# Thai sentence segmentation
from thai_segmenter import sentence_segment

# Thai word tokenization
import spacy_thai
tokenizer = spacy_thai.load()

In [ ]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text
    
    https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py"""
    
    def __init__(self, tokenizer):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        self.tokenizer = tokenizer
        self.tokens = []
    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        # for word in STOP_WORDS.union(set(stopwords)):
        #     lexeme = nlp.vocab[word]
        #     lexeme.is_stop = True
        pass
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = self.tokenizer(text)
        self.tokens = doc
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
# example text
paragraph = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

tr4w = TextRank4Keyword( tokenizer=tokenizer )
tr4w.analyze(paragraph, window_size=4, lower=False)
tr4w.get_keywords(10)

หุ่นยนต์ - 3.8208226410979584
ฟี - 3.2010214282201805
โบ้ - 3.0128038212930957
ระดับ - 2.1845160081093384
วิทยาการ - 1.9744561722570322
ไทย - 1.8902690409696246
ระบบ - 1.5975011946998237
สถาบัน - 1.5255719750453376
การผลิต - 1.4459418582161931
ธุรกิจ - 1.4314885609776313
ชั้นนำ - 1.3453801323545687
บัณฑิต - 1.3289964457387926


In [ ]:
len(tr4w.tokens), tr4w.tokens[0]

(234, แนวโน้ม)

In [ ]:
len(tr4w.node_weight), tr4w.node_weight

(78,
 {'แนวโน้ม': 0.5151560838054188,
  'หุ่นยนต์': 3.8208226410979584,
  'การผลิต': 1.4459418582161931,
  'งาน': 0.8537608817852167,
  'จำนวน': 0.8771629950228563,
  '%': 0.8514785999020578,
  'ทุกปี': 0.7997866474831316,
  'ประเทศ': 0.7696493424149055,
  'ไทย': 1.8902690409696246,
  'บุคลากร': 0.7672077277443364,
  'วิทยาการ': 1.9744561722570322,
  'ระบบ': 1.5975011946998237,
  'ระดับ': 2.1845160081093384,
  'อาชีพ': 0.7400156816748369,
  'วิศวกร': 0.7369885981671944,
  'อาจารย์': 0.7324480662206776,
  'นักวิจัย': 0.7280913862541138,
  'ฟี': 3.2010214282201805,
  'โบ้': 3.0128038212930957,
  'ปี': 0.7628582280592622,
  'รศ.': 1.071189796431153,
  'ดร.': 1.190692022997896,
  'ชิต': 0.7481406354666695,
  'วัฒนา': 0.7628582280592622,
  'คติพจน์': 0.8129225541155237,
  '“': 0.8602456370958648,
  'Cradle': 0.8873064516129031,
  'of': 0.9186039833822092,
  'Future': 0.9374777486237588,
  'Leaders': 0.9396269768282713,
  'in': 0.9119761208576997,
  'Robotics': 0.8760612484494064,
  '”': 0.8